In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.externals import joblib
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
data = joblib.load('data.pkl')

In [3]:
data_dir = 'bbdc_2019_Bewegungsdaten/'

In [4]:
data.keys()

dict_keys(['features', 'labels', 'bounds', 'lens', 'label_encoder'])

In [5]:
def standard_scale(data):
    features = data['features']
    lens = np.concatenate((data['lens']['train'],data['lens']['valid'],data['lens']['test']))
    features = np.concatenate(features, axis=0)
    features = StandardScaler().fit_transform(features)
    ret = []
    l = 0
    for t in lens:
        ret.append(features[l: l + t])
        l += t
    features = np.array(ret)
    data['features'] = features

In [6]:
standard_scale(data)

In [7]:
def get_split(data, split):
    if split == 'train':
        return data['features'][: data['bounds']['train']]
    elif split == 'valid':
        return data['features'][data['bounds']['train']: data['bounds']['train'] + data['bounds']['valid']]
    elif split == 'test':
        return data['features'][data['bounds']['train'] + data['bounds']['valid']:]

In [8]:
def generate_batch_idx(n, batch_size, randomise=False):
    idx = np.arange(0, n)
    if randomise:
        np.random.shuffle(idx)
    for batch_idx in np.arange(0, n, batch_size):
        yield idx[batch_idx:batch_idx+batch_size]

In [9]:
def generate_batches(data, split, batch_size,
                     skip=5, randomise=False):
    features = get_split(data, split)

    try:
        labels = data['label_encoder'].transform(data['labels'][split])
    except:
        labels = np.zeros(features.shape[0])
    
    lens = data['lens'][split]
    new_features = []
    new_labels = []
    new_lens = []
    for i in range(len(lens)):
        mat = features[i]
        label = labels[i]
        l = lens[i]
        idx = np.arange(0, mat.shape[0], skip)
        new_features.append(mat[idx,:])
        new_labels.append(label)
        new_lens.append(idx.shape[0])
    
    features = np.array(new_features)
    labels = np.array(new_labels)
    lens = np.array(new_lens)
    
    n = len(features)
    for batch_idx in generate_batch_idx(n, batch_size, randomise):
        batch_data = features[batch_idx]
        batch_labels = labels[batch_idx]
        batch_lens = lens[batch_idx]
        yield batch_data, batch_labels, batch_lens

In [10]:
# def sort_batch(batch, targets, lengths):
#     """
#     Sort a minibatch by the length of the sequences with the longest sequences first
#     return the sorted batch targes and sequence lengths.
#     This way the output can be used by pack_padded_sequences(...)
#     """
#     perm_idx = np.argsort(lengths)[::-1]
#     seq_lengths = lengths[perm_idx]
#     seq_tensor = batch[perm_idx]
#     target_tensor = targets[perm_idx]
#     return seq_tensor, target_tensor, seq_lengths

def pad_batch(batch, lens):
    max_len = max(lens)
    batch_size = batch.shape[0]
    num_feature = batch[0].shape[1]
    padded_seqs = np.zeros((batch_size, max_len, num_feature))
    
    for i, l in enumerate(lens):
        padded_seqs[i, :l, :] = batch[i][:l]

    return padded_seqs

In [11]:
def torch_batch(batch, targets):
    return torch.from_numpy(batch).float(), torch.from_numpy(targets).long()

In [12]:
def get_preds(model, data, split, batch_size, stride):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for b_data, b_labels, b_lens in generate_batches(data, split, batch_size, 
                                                         stride, False):
            b_data = pad_batch(b_data, b_lens)
            b_data, b_labels = torch_batch(b_data, b_labels)
            preds.append(model(b_data, b_lens))
            labels.append(b_labels)
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)
    return preds, labels

In [13]:
def get_accuracy(preds, labels, le):
    preds = preds.max(dim=1)[1].numpy()
    preds = [le.classes_[i] for i in preds]
    labels = labels.numpy()
    labels = [le.classes_[i] for i in labels]
    return accuracy_score(labels, preds)

In [37]:
class HARNet(nn.Module):
    def __init__(self, time_steps):
        super().__init__()
        self.rnn = nn.GRU(14, 64)
        self.lin2 = nn.Linear(64, 64, bias=True)
        self.lin3 = nn.Linear(64, 22, bias=True)
    
    def forward(self, data, lens):
        x, _ = self.rnn(data[:, :, 5:])
        x = torch.cat([seq[l-1].view(1, -1) for l, seq in zip(lens, x)], dim=0)
        x = self.lin2(x)
        x = F.relu(x)
        x = self.lin3(x)
        return x

In [39]:
num_epochs = 10000
batch_size = 32
stride = 5
objective = nn.CrossEntropyLoss()
model = HARNet(time_steps)
optimiser = torch.optim.Adam(model.parameters(), weight_decay=0.0)
running_loss = 0
running_batch = 0
min_valid_loss = float('inf')

for epoch in range(1, num_epochs + 1):
    with tqdm(enumerate(generate_batches(data, 'valid', batch_size,
                                         stride, True), 1)) as pbar:
        model.train()
        for batch_num, (batch_data, batch_labels, batch_lens) in pbar:
            batch_data = pad_batch(batch_data, batch_lens)
            batch_data, batch_labels = torch_batch(batch_data, batch_labels)
            optimiser.zero_grad()
            preds = model(batch_data, batch_lens)
            loss = objective(preds, batch_labels)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 2, norm_type=2)
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
            running_batch += 1
            pbar.set_description(f'[Epoch: {epoch}] | Batch {batch_num} | Loss: {running_loss/running_batch}')
            
    valid_preds, valid_labels = get_preds(model, data, 'valid',
                                          64, stride)
    
    valid_loss = objective(valid_preds, valid_labels).item()
    
    if valid_loss < min_valid_loss:
        print(f'Validation loss improved from {min_valid_loss} to {valid_loss}')
        acc = get_accuracy(valid_preds, valid_labels, data['label_encoder'])
        print(f'Validation accuracy: {acc}')
        min_valid_loss = valid_loss
        with open('best_rnn_model.pt', 'wb') as f:
            torch.save(model.state_dict(), f)
    else:
        print('Validation loss did not improve')

[Epoch: 1] | Batch 36 | Loss: 3.073989040321774: : 36it [00:19,  1.89it/s] 
0it [00:00, ?it/s]

Validation loss improved from inf to 3.029745101928711
Validation accuracy: 0.09849157054125998


[Epoch: 2] | Batch 36 | Loss: 3.0157430701785617: : 36it [00:18,  1.99it/s]
0it [00:00, ?it/s]

Validation loss improved from 3.029745101928711 to 2.8421804904937744
Validation accuracy: 0.12244897959183673


[Epoch: 3] | Batch 36 | Loss: 2.938975621152807: : 36it [00:17,  2.02it/s] 
0it [00:00, ?it/s]

...

Validation loss did not improve


[Epoch: 44] | Batch 36 | Loss: 1.8179723211009093: : 36it [00:27,  1.71it/s]
0it [00:00, ?it/s]

Validation loss improved from 1.2127554416656494 to 1.175963044166565
Validation accuracy: 0.5980479148181012


[Epoch: 45] | Batch 9 | Loss: 1.814711975849281: : 9it [00:05,  1.49it/s] 


KeyboardInterrupt: 

In [ ]:
np.arange(0, 6000, 10).shape